In [55]:
#Imports das bibliotecas

import pandas as pd
import numpy as np
import seaborn as sns 
import os
import glob


Visualizar dados de uma coluna do dataframe --> final[final['Origem_Aba'] == 'true_label']

Remover coluna de um dataframe --> final = final.drop(columns=['Unnamed: 0'])



In [70]:
pyt1 = pd.read_excel('./vulnerable_files/labels/labels.xlsx', engine='openpyxl', sheet_name=None)
merged = []
special_tabs = ['codeql', 'bandit']

for tab_name, df in pyt1.items():

    if tab_name in special_tabs and 'label' in df.columns:
        df['Vulline'] = df['label']
        df = df.drop(columns=['label'])

    # Cria uma nova coluna com o nome da aba atual
    df['Sast_tool'] = tab_name
    merged.append(df)

df_pyt1 = pd.concat(merged, ignore_index=True)

df_pyt1 = df_pyt1.drop(columns=['Unnamed: 0'])


In [68]:
pyt2 = pd.read_excel('./vulnerable_files/labels/PyTy2_final_labels.xlsx', engine='openpyxl')

df_pyt2 = pd.DataFrame()
df_pyt2['Filename'] = pyt2['Filename']
df_pyt2['Vulline'] = np.nan

colunas_sast = ['Mr. Chekideh', 'Semgrep', 'Bandit']

df_pyt2 = pyt2.melt(id_vars=['Filename'] ,value_vars=colunas_sast, var_name='Sast_tool', value_name='Vulline')

df_pyt2['Sast_tool'] = df_pyt2['Sast_tool'].replace("Mr. Chekideh", "true_label")


In [67]:
siddiq = pd.read_excel('./vulnerable_files/labels/siddiq_final_labels.xlsx', engine='openpyxl')

df_siddiq = pd.DataFrame()
df_siddiq['Filename'] = pyt2['Filename']
df_siddiq['Vulline'] = np.nan

colunas_sast = ['Mr. Chekideh', 'Semgrep', 'Bandit']

df_siddiq = siddiq.melt(id_vars=['Filename'] ,value_vars=colunas_sast, var_name='Sast_tool', value_name='Vulline')

df_siddiq['Sast_tool'] = df_siddiq['Sast_tool'].replace("Mr. Chekideh", "true_label")


In [73]:
df_final = pd.concat(objs=[df_pyt1, df_pyt2, df_siddiq])

df_final.tail()

,Filename,Vulline,Sast_tool
385,CWE-841_mitre_1.py,NaN,Bandit
386,CWE-918_codeql_1.py,13(CWE-400),Bandit
387,CWE-918_codeql_2.py,13(CWE-400),Bandit
388,CWE-941_mitre_1.py,NaN,Bandit
389,CWE-943_sonar_1.py,NaN,Bandit
